# Introduction
This file will be used to format the training set for the different tests 

In [ ]:
%reset -f
import os
import shutil
import random
from tqdm import tqdm  # For the progress bar

# Path to the 'dataset' folder
dataset_folder = "dataset"

# List of folders to keep
folders_to_keep = ["images", "labels"]

# Iterate through all items in the dataset folder
for item in os.listdir(dataset_folder):
    item_path = os.path.join(dataset_folder, item)
    
    # Check if the item is not in the keep list
    if item not in folders_to_keep:
        # Remove the folder or file
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)  # Remove directories
            print(f"Removed folder: {item_path}")
        else:
            os.remove(item_path)  # Remove files
            print(f"Removed file: {item_path}")



# Image reordering
Copies all the images from ~/datasets/coco into the /dataset folder

In [ ]:
%reset -f
import os
import shutil
from tqdm import tqdm

# Define source directories
coco_train_images = os.path.expanduser("~/datasets/coco/images/train2017")
coco_val_images = os.path.expanduser("~/datasets/coco/images/val2017")
coco_train_labels = os.path.expanduser("~/datasets/coco/labels/train2017")
coco_val_labels = os.path.expanduser("~/datasets/coco/labels/val2017")

# Define target directories
dataset_images = "dataset/images"
dataset_labels = "dataset/labels"

# Ensure target directories exist
os.makedirs(dataset_images, exist_ok=True)
os.makedirs(dataset_labels, exist_ok=True)

# Helper function to copy files
def copy_files(source_dir, target_dir, file_extension, phase_name):
    """
    Copy files with a specific extension from source_dir to target_dir.

    Args:
        source_dir (str): Source directory path.
        target_dir (str): Target directory path.
        file_extension (str): File extension to filter.
        phase_name (str): Name of the phase (e.g., 'train', 'val') for progress.
    """
    files = [f for f in os.listdir(source_dir) if f.endswith(file_extension)]
    
    with tqdm(total=len(files), desc=f"Copying {phase_name} files", unit="file") as pbar:
        for file_name in files:
            src_path = os.path.relpath(os.path.join(source_dir, file_name))
            dst_path = os.path.relpath(os.path.join(target_dir, file_name))
            shutil.copy(src_path, dst_path)
            pbar.update(1)

# Copy files
copy_files(coco_train_images, dataset_images, ".jpg", "Training Images")
copy_files(coco_val_images, dataset_images, ".jpg", "Validation Images")
copy_files(coco_train_labels, dataset_labels, ".txt", "Training Labels")
copy_files(coco_val_labels, dataset_labels, ".txt", "Validation Labels")

print("Files successfully copied to dataset/images and dataset/labels with relative paths.")




Reformat dataset to include only vehicles.
The original dataset contains the labels of:

    ["aeroplane", "bicyclebike", "bird", "boat", "bottle", "bus",
    "car", "cat", "chair", "cow", "diningtable", "dog", "horse",
    "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

A new filtered label directory has been created so that only remain the vehicles:

    ["car", "bus", "motorbike", "bicyclebike"]

These new labels are stored in dataset/labels_filtered

In [ ]:
# Directory containing YOLO label .txt files
label_dir = "dataset/labels"  # Replace with your label directory path

# Allowed Class IDs for vehicle-related objects
ALLOWED_CLASSES = {0, 1, 2, 3, 4, 5, 6, 7}  # person, bicycle, car, motorcycle, airplane, bus, train, truck

# Directory to save filtered labels
output_dir = "dataset/labels_filtered"
os.makedirs(output_dir, exist_ok=True)

def filter_labels(label_file):
    """
    Reads a YOLO label file, filters out unwanted classes,
    and writes the remaining labels to a new file.
    """
    input_path = os.path.join(label_dir, label_file)
    output_path = os.path.join(output_dir, label_file)

    with open(input_path, "r") as infile, open(output_path, "w") as outfile:
        for line in infile:
            parts = line.split()
            class_id = int(parts[0])  # Extract class ID
            if class_id in ALLOWED_CLASSES:
                # Write the line if class ID is allowed
                outfile.write(line)

# List all .txt files in the label directory
label_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]

# Process all .txt files with a progress bar
with tqdm(total=len(label_files), desc="Filtering Labels", unit="file") as pbar:
    for file_name in label_files:
        filter_labels(file_name)
        pbar.update(1)

print(f"Filtered labels saved in: {output_dir}")

Remove empty labels

In [ ]:
# Define the path to the labels folder
labels_folder = "dataset/labels_filtered"

# List all .txt files in the labels folder
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Initialize a counter for removed files
removed_count = 0

# Check each label file and remove it if it's empty
for label_file in label_files:
    label_path = os.path.join(labels_folder, label_file)
    if os.path.getsize(label_path) == 0:  # Check if the file size is 0 bytes
        os.remove(label_path)  # Remove the empty file
        removed_count += 1
        # print(f"Removed empty label: {label_file}")

# Output the result
print(f"Total empty labels removed: {removed_count}")



Counting images and filtered_labels

In [ ]:
print(os.getcwd())
# Define the paths to the images and labels folders
images_folder = "dataset/images"
labels_folder = "dataset/labels_filtered"

# List all files in the images and labels folders
image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Count the total number of images and labels
num_images = len(image_files)
num_labels = len(label_files)

# Check for matching files (base filenames without extensions)
image_basenames = {os.path.splitext(f)[0] for f in image_files}
label_basenames = {os.path.splitext(f)[0] for f in label_files}

# Count matched and unmatched files
matched_files = image_basenames & label_basenames
unmatched_images = image_basenames - label_basenames
unmatched_labels = label_basenames - image_basenames

print(f"Total images: {num_images}")
print(f"Total labels: {num_labels}")
print(f"Matched files: {len(matched_files)}")
print(f"Unmatched images: {len(unmatched_images)}")
print(f"Unmatched labels: {len(unmatched_labels)}")

# Optionally print the unmatched files
if unmatched_images:
    print("Unmatched images (no corresponding label):")
    for img in unmatched_images:
        print(f"  {img}")

if unmatched_labels:
    print("Unmatched labels (no corresponding image):")
    for lbl in unmatched_labels:
        print(f"  {lbl}")


Create the folder of images_filtered with a reduced number of unlabellel images.
The ratio of labelled images and unlabelled images has been set to 50/50. 

In [ ]:
import random

# Paths
images_folder = "dataset/images"
labels_folder = "dataset/labels_filtered"
output_folder = "dataset/images_filtered"

# Ratio of labeled and unlabeled images
r_label = 50
r_unlabel = 100 - r_label

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Get all image files and corresponding label files
image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Get the base filenames (without extensions) for labels
label_basenames = {os.path.splitext(label)[0] for label in label_files}

# Separate labeled and unlabeled images
labeled_images = [img for img in image_files if os.path.splitext(img)[0] in label_basenames]
unlabeled_images = [img for img in image_files if os.path.splitext(img)[0] not in label_basenames]

# Check counts
num_labeled = len(labeled_images)
num_unlabeled_to_select = min(int(num_labeled * r_unlabel / r_label), len(unlabeled_images))

# Randomly select the required number of unlabeled images
selected_unlabeled_images = random.sample(unlabeled_images, num_unlabeled_to_select)

# Combine labeled and selected unlabeled images
images_to_copy = labeled_images + selected_unlabeled_images

# Copy labeled and selected unlabeled images to the output folder with a progress bar
with tqdm(total=len(images_to_copy), desc="Copying Images", unit="file") as pbar:
    for img in images_to_copy:
        src_path = os.path.join(images_folder, img)
        dst_path = os.path.join(output_folder, img)
        shutil.copy(src_path, dst_path)
        pbar.update(1)

# Output results
print(f"Total labeled images: {num_labeled}")
print(f"Total unlabeled images selected: {len(selected_unlabeled_images)}")
print(f"Total images in 'images_filtered': {len(os.listdir(output_folder))}")



# Create Train, Validation and Test image sets
From the image and labels ("dataset/images", "dataset/labels_filtered")
Create the test, validation and test sets.


- Training is stored in ("dataset/train/images", "dataset/train/labels")
- Validation is stored in ("dataset/valid/images", "dataset/valid/labels")
- Test is stored in ("dataset/test/images", "dataset/test/labels")


In [ ]:
import os
import shutil
import random
from tqdm import tqdm  # For progress bar

The following code takes the unfiltered images and create the corresponding training, validation and test set in the following folders:
- dataset/train
- dataset/valid
- dataset/test

In [ ]:
# Split data into 60% train, 20% validation, 20% test
train_perc = .6
valid_perc = .2

# Define folder paths
images_folder = "dataset/images"  # Folder containing filtered images
labels_folder = "dataset/labels"  # Folder containing filtered labels

train_images_folder = "dataset/train/images"
train_labels_folder = "dataset/train/labels"

valid_images_folder = "dataset/valid/images"
valid_labels_folder = "dataset/valid/labels"

test_images_folder = "dataset/test/images"
test_labels_folder = "dataset/test/labels"

# Create output directories
for folder in [train_images_folder, train_labels_folder,
               valid_images_folder, valid_labels_folder,
               test_images_folder, test_labels_folder]:
    os.makedirs(folder, exist_ok=True)

# Get a list of all images
image_files = sorted(os.listdir(images_folder))

# Create a list of images with and without labels
data = []
for image_file in image_files:
    label_file = os.path.splitext(image_file)[0] + ".txt"
    if os.path.exists(os.path.join(labels_folder, label_file)):
        data.append((image_file, label_file))  # Image has a corresponding label
    else:
        data.append((image_file, None))  # Image has no label (no objects detected)

# Shuffle the data
random.shuffle(data)

# Split data into ratios
train_split = int(train_perc * len(data))
valid_split = int((train_perc + valid_perc) * len(data))

train_data = data[:train_split]
valid_data = data[train_split:valid_split]
test_data = data[valid_split:]

# Function to copy images and labels with a progress bar
def copy_files(data, dest_images_folder, dest_labels_folder, phase_name):
    with tqdm(total=len(data), desc=f"Copying {phase_name}") as pbar:
        for image_file, label_file in data:
            # Copy the image file
            shutil.copy(os.path.join(images_folder, image_file), os.path.join(dest_images_folder, image_file))
            # Copy the label file if it exists
            if label_file:
                shutil.copy(os.path.join(labels_folder, label_file), os.path.join(dest_labels_folder, label_file))
            # Update progress bar
            pbar.update(1)

# Copy data to respective folders
copy_files(train_data, train_images_folder, train_labels_folder, "Training Data")
copy_files(valid_data, valid_images_folder, valid_labels_folder, "Validation Data")
copy_files(test_data, test_images_folder, test_labels_folder, "Testing Data")

print("Dataset split complete!")
print(f"Training data: {len(train_data)} images")
print(f"Validation data: {len(valid_data)} images")
print(f"Testing data: {len(test_data)} images")


The following code takes the filtered images and create the corresponding training, validation and test set in the following folders:
- dataset/train_filtered
- dataset/valid_filtered
- dataset/test_filtered


In [ ]:
# Split data into 60% train, 20% validation, 20% test
train_perc = .6
valid_perc = .2

# Define folder paths
images_folder = "dataset/images_filtered"  # Folder containing filtered images
labels_folder = "dataset/labels_filtered"  # Folder containing filtered labels

train_images_folder = "dataset/train_filtered/images"
train_labels_folder = "dataset/train_filtered/labels"

valid_images_folder = "dataset/valid_filtered/images"
valid_labels_folder = "dataset/valid_filtered/labels"

test_images_folder = "dataset/test_filtered/images"
test_labels_folder = "dataset/test_filtered/labels"

# Create output directories
for folder in [train_images_folder, train_labels_folder,
               valid_images_folder, valid_labels_folder,
               test_images_folder, test_labels_folder]:
    os.makedirs(folder, exist_ok=True)

# Get a list of all images
image_files = sorted(os.listdir(images_folder))

# Create a list of images with and without labels
data = []
for image_file in image_files:
    label_file = os.path.splitext(image_file)[0] + ".txt"
    if os.path.exists(os.path.join(labels_folder, label_file)):
        data.append((image_file, label_file))  # Image has a corresponding label
    else:
        data.append((image_file, None))  # Image has no label (no objects detected)

# Shuffle the data
random.shuffle(data)

# Split data into ratios
train_split = int(train_perc * len(data))
valid_split = int((train_perc + valid_perc) * len(data))

train_data = data[:train_split]
valid_data = data[train_split:valid_split]
test_data = data[valid_split:]

# Function to copy images and labels with a progress bar
def copy_files(data, dest_images_folder, dest_labels_folder, phase_name):
    with tqdm(total=len(data), desc=f"Copying {phase_name}") as pbar:
        for image_file, label_file in data:
            # Copy the image file
            shutil.copy(os.path.join(images_folder, image_file), os.path.join(dest_images_folder, image_file))
            # Copy the label file if it exists
            if label_file:
                shutil.copy(os.path.join(labels_folder, label_file), os.path.join(dest_labels_folder, label_file))
            # Update progress bar
            pbar.update(1)

# Copy data to respective folders
copy_files(train_data, train_images_folder, train_labels_folder, "Training Data")
copy_files(valid_data, valid_images_folder, valid_labels_folder, "Validation Data")
copy_files(test_data, test_images_folder, test_labels_folder, "Testing Data")

print("Dataset split complete!")
print(f"Training data: {len(train_data)} images")
print(f"Validation data: {len(valid_data)} images")
print(f"Testing data: {len(test_data)} images")



In [ ]:
import os
from collections import defaultdict
from tqdm import tqdm

# Directories to process
directories = [
    "dataset/labels",
    "dataset/train/labels",
    "dataset/valid/labels",
    "dataset/test/labels",
    
    "dataset/labels_filtered",
    "dataset/train_filtered/labels",
    "dataset/valid_filtered/labels",
    "dataset/test_filtered/labels",
]

def count_labels_in_directory(directory):
    """
    Counts the number of each class in a directory of YOLO label files with a progress bar.
    
    Args:
        directory (str): Path to the directory containing label files.
        
    Returns:
        dict: A dictionary with class IDs as keys and counts as values.
    """
    class_counts = defaultdict(int)
    
    # Get all .txt files in the directory
    label_files = [f for f in os.listdir(directory) if f.endswith(".txt")]
    
    # Process each file with a progress bar
    with tqdm(total=len(label_files), desc=f"Processing {directory}", unit="file") as pbar:
        for label_file in label_files:
            label_path = os.path.join(directory, label_file)
            with open(label_path, "r") as file:
                for line in file:
                    parts = line.split()
                    class_id = int(parts[0])  # Extract class ID
                    class_counts[class_id] += 1  # Increment the count for the class ID
            pbar.update(1)
    
    return class_counts

# Count labels for each directory
results = {}

for directory in directories:
    if os.path.exists(directory):
        results[directory] = count_labels_in_directory(directory)
    else:
        results[directory] = None  # Directory does not exist

# Display the results
print("\nClass counts per directory:")
for directory, class_counts in results.items():
    print(f"\nDirectory: {directory}")
    if class_counts is not None:
        for class_id, count in sorted(class_counts.items()):
            print(f"  Class {class_id}: {count}")
    else:
        print("  Directory does not exist or contains no labels.")



# Load YOLO Model and Begin Training!!

In [2]:
from ultralytics import YOLO

In [3]:
model16 = YOLO("yolo11n.yaml")
model16.train(
    data="train_filtered.yaml",
    project="./filtered_from_scratch_16",
    pretrained=False,  
    epochs = 100, 
    # patience=10, 
    save_period=10,
    batch=16,
    resume=True, 
    save=True
    )

New https://pypi.org/project/ultralytics/8.3.55 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.54 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.yaml, data=train_filtered.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=./filtered_from_scratch_16, name=train5, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=None, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=

train: Scanning /home/tyche/ForeHelm/YOLO_training/COCO/dataset/train_filtered/labels.cache... 45916 images, 28056 backgrounds, 0 corrupt: 100%|██████████| 73972/73972 [00:00<?, ?it/s]
val: Scanning /home/tyche/ForeHelm/YOLO_training/COCO/dataset/valid_filtered/labels.cache... 15296 images, 9361 backgrounds, 0 corrupt: 100%|██████████| 24657/24657 [00:00<?, ?it/s]


Plotting labels to filtered_from_scratch_16/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to filtered_from_scratch_16/train5
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.88G      3.044      4.078      3.439         15        640: 100%|██████████| 4624/4624 [13:00<00:00,  5.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 771/771 [02:17<00:00,  5.60it/s]


: 

In [ ]:
modelX = YOLO("yolo11n.yaml")
modelX.train(
    data="train_filtered.yaml",
    project="./filtered_from_scratch_X",
    pretrained=False,  
    epochs = 100, 
    patience=10, 
    save_period=10,
    batch=-1,
    resume=True, 
    save=True
    )

# Validating the YOLO models
The following code is used to validate and compare the different YOLO models that have been trained

This code tries the default and pretrained yolo11n model with the COCO dataset

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")

# Customize validation settings
validation_results = model.val(data="coco.yaml", imgsz=640, batch=16, conf=0.25, iou=0.6, device="0", plots=True)

In [ ]:
# Load a model
model = YOLO("yolo11n.pt")

# Customize validation settings
validation_results = model.val(data="coco.yaml", imgsz=640, batch=16, conf=0.25, iou=0.6, device="0", plots=True)

In [ ]:
# Validate the model
validation_results.box.map  # map50-95
validation_results.box.map50  # map50
validation_results.box.map75  # map75
validation_results.box.maps  # a list contains map50-95 of each category